In [1]:
# requirements

import pandas as pd
import numpy as np
import requests
import json
import nltk
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import math

In [2]:
# to communicate with google spreadsheet...
import gspread
from gspread_dataframe import get_as_dataframe
from gspread_dataframe import set_with_dataframe
from google.oauth2 import service_account # based on google-auth library
import sddk

s = sddk.cloudSession("sciencedata.dk")
# establish connection with gogglesheets...
file_data = s.read_file("https://sciencedata.dk/files/ServiceAccountsKey.json", "dict") # or load it from a local storage: json.load(open("../../ServiceAccountsKey.json", "r"))
credentials = service_account.Credentials.from_service_account_info(file_data)
gc = gspread.Client(auth=credentials.with_scopes(['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']))
PIPA_data = gc.open_by_url("https://docs.google.com/spreadsheets/d/1rV4t0_UV_wcx--UAHVwkqB8Wa_5n9mnpV05yGG1OHqk/edit?usp=sharing")

endpoint variable has been configured to: https://sciencedata.dk/files/


In [3]:
cgl = pd.read_json("../data/large_data/cgl_repl.json")

In [4]:
cgl.head(10)

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,tlg_epithet,clean_string,n_sentences,lemmatized_sentences,lemmata,lemmata_wordcount,subcorpus,lemmata_repl,lemmatized_sentences_repl
1,tlg0003.tlg001.perseus-grc2.xml,Thucydides,The Peloponnesian War,150118,tlg0003,tlg0003.tlg001,5 B.C.,-4.5,{'-4.5': 1},-4.5,pagan,Historici/-ae,Θουκυδίδης Ἀθηναῖος ξυνέγραψε τὸν πόλεμον τῶν...,6068,"[[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελ...","[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελο...",71863,None,"[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελο...","[[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελ..."
6,tlg0006.tlg001.perseus-grc2.xml,Euripides,Cyclops,4141,tlg0006,tlg0006.tlg001,5 B.C.,-4.5,{'-4.5': 1},-4.5,pagan,Tragici,"Ὦ Βρόμιε, διὰ σὲ μυρίους ἔχω πόνους νῦν χὥτʼ ἐ...",529,"[[Βρόμιος, ἔχω, πόνος, χὥτʼ, ἥβη, ἐμός, εὐσθεν...","[Βρόμιος, ἔχω, πόνος, χὥτʼ, ἥβη, ἐμός, εὐσθενέ...",2535,None,"[Βρόμιος, ἔχω, πόνο*, χὥτʼ, ἥβη, ἐμός, εὐσθενέ...","[[Βρόμιος, ἔχω, πόνο*, χὥτʼ, ἥβη, ἐμός, εὐσθεν..."
7,tlg0006.tlg004.perseus-grc2.xml,Euripides,Ἡρακλεῖδαι,6272,tlg0006,tlg0006.tlg004,5 B.C.,-4.5,{'-4.5': 1},-4.5,pagan,Tragici,Πάλαι ποτʼ ἐστὶ τοῦτʼ ἐμοὶ δεδογμένον· ὁ μὲν ...,662,"[[ποτός, εἰμί, οὗτος, δεδογμένον], [δίκαιος, φ...","[ποτός, εἰμί, οὗτος, δεδογμένον, δίκαιος, φύω,...",3545,None,"[ποτός, εἰμί, οὗτος, δεδογμένον, δίκαιος, φύω,...","[[ποτός, εἰμί, οὗτος, δεδογμένον], [δίκαιος, φ..."
8,tlg0006.tlg005.perseus-grc2.xml,Euripides,Ἱππόλυτος,8257,tlg0006,tlg0006.tlg005,5 B.C.,-4.5,{'-4.5': 1},-4.5,pagan,Tragici,"Πολλὴ μὲν ἐν βροτοῖσι κοὐκ ἀνώνυμος, θεὰ κέκλ...",918,"[[πολύς, βροτός, ἀνώνυμος, θεά, καλέω, Κύπρις,...","[πολύς, βροτός, ἀνώνυμος, θεά, καλέω, Κύπρις, ...",4898,None,"[πολύς, βροτός, ἀνώνυμος, θεά, καλέω, Κύπρις, ...","[[πολύς, βροτός, ἀνώνυμος, θεά, καλέω, Κύπρις,..."
9,tlg0006.tlg006.perseus-grc2.xml,Euripides,Ἀνδρομάχη,7397,tlg0006,tlg0006.tlg006,5 B.C.,-4.5,{'-4.5': 1},-4.5,pagan,Tragici,"Ἀσιάτιδος γῆς σχῆμα, Θηβαία πόλι, ὅθεν ποθʼ ἕ...",712,"[[Ἀσιανός, γῆ, σχῆμα, θηβαία, πόλις, ἑδνόω, πο...","[Ἀσιανός, γῆ, σχῆμα, θηβαία, πόλις, ἑδνόω, πολ...",4420,None,"[Ἀσιανός, γῆ, σχῆμα, θηβαία, πόλις, ἑδνόω, πολ...","[[Ἀσιανός, γῆ, σχῆμα, θηβαία, πόλις, ἑδνόω, πο..."
10,tlg0006.tlg007.perseus-grc2.xml,Euripides,Ἑκάβη,7340,tlg0006,tlg0006.tlg007,5 B.C.,-4.5,{'-4.5': 1},-4.5,pagan,Tragici,"Ἥκω νεκρῶν κευθμῶνα καὶ σκότου πύλας λιπών, ἵν...",754,"[[ἥκω, νεκρός, κευθμών, σκότος, πύλη, λιπάω, ᾍ...","[ἥκω, νεκρός, κευθμών, σκότος, πύλη, λιπάω, ᾍδ...",4422,None,"[ἥκω, νεκρός, κευθμών, σκότος, πύλη, λιπάω, ᾍδ...","[[ἥκω, νεκρός, κευθμών, σκότος, πύλη, λιπάω, ᾍ..."
11,tlg0006.tlg008.perseus-grc2.xml,Euripides,Ἱκέτιδες,7166,tlg0006,tlg0006.tlg008,5 B.C.,-4.5,{'-4.5': 1},-4.5,pagan,Tragici,"Δήμητερ ἑστιοῦχʼ Ἐλευσῖνος χθονὸς τῆσδʼ, οἵ τ...",721,"[[Δήμητηρ, ἑστιοῦχος, Ἐλευσίς, χθών, ναός, ἔχω...","[Δήμητηρ, ἑστιοῦχος, Ἐλευσίς, χθών, ναός, ἔχω,...",4402,None,"[Δήμητηρ, ἑστιοῦχος, Ἐλευσίς, χθών, ναός, ἔχω,...","[[Δήμητηρ, ἑστιοῦχος, Ἐλευσίς, χθών, ναός, ἔχω..."
12,tlg0006.tlg009.perseus-grc2.xml,Euripides,Ἡρακλῆς,7977,tlg0006,tlg0006.tlg009,5 B.C.,-4.5,{'-4.5': 1},-4.5,pagan,Tragici,"Τίς τὸν Διὸς σύλλεκτρον οὐκ οἶδεν βροτῶν, Ἀργ...",842,"[[Ζεύς, σύλλεκτρος, οἶδα, βροτός, Ἀργεῖος, ἀμφ...","[Ζεύς, σύλλεκτρος, οἶδα, βροτός, Ἀργεῖος, ἀμφι...",4936,None,"[Ζεύς, σύλλεκτρος, οἶδα, βροτός, Ἀργεῖος, ἀμφι...","[[Ζεύς, σύλλεκτρος, οἶδα, βροτός, Ἀργεῖος, ἀμφ..."
13,tlg0006.tlg010.perseus-grc2.xml,Euripides,Ἴων,9339,tlg0006,tlg0006.tlg010,5 B.C.,-4.5,{'-4.5': 1},-4.5,pagan,Tragici,"Ἄτλας, ὁ χαλκέοισι νώτοις οὐρανὸν θεῶν παλαιὸ...",1090,"[[Ἄτλας, χαλκοῦς, νῶτον, οὐρανός, θεός, παλαιό...","[Ἄτλας, χαλκοῦς, νῶτον, οὐρανός, θεός, παλαιός...",5643,None,"[Ἄτλας, χαλκοῦς, νῶτον, οὐρανός, θεός, παλαιός...","[[Ἄτλας, χαλκοῦς, νῶτον, οὐρανός, θεός, παλαιό..."
14,tlg0006.tlg011.perseus-grc2.xml,Euripides,Τρῳάδες,7230,tlg0006,tlg0006.tlg011,5 B.C.,-4.5,{'-4.5': 1},-4.5,pagan,Tragici,Ἥκω λιπὼν Αἴγαιον ἁλμυρὸν βάθος πόντου Ποσειδ.

In [5]:
# greek dictionary - used for automatic translations
morpheus_by_lemma = json.loads(requests.get("https://sciencedata.dk/public/8fe7d59de1eafe5f8eaebc0044534606/morpheus_by_lemma.json").content)

In [6]:
key_words = ['πόνο*', 'ὀδύν*', 'ἄλγ*', 'λύπ*']

In [7]:
for keyword in key_words:
    cgl["count_" + keyword] = cgl["lemmata_repl"].apply(lambda x: x.count(keyword))

In [18]:
cgl.head(10)

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,...,lemmatized_sentences,lemmata,lemmata_wordcount,subcorpus,lemmata_repl,lemmatized_sentences_repl,count_πόνο*,count_ὀδύν*,count_ἄλγ*,count_λύπ*
1,tlg0003.tlg001.perseus-grc2.xml,Thucydides,The Peloponnesian War,150118,tlg0003,tlg0003.tlg001,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελ...","[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελο...",71863,None,"[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελο...","[[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελ...",31,0,6,25
6,tlg0006.tlg001.perseus-grc2.xml,Euripides,Cyclops,4141,tlg0006,tlg0006.tlg001,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[[Βρόμιος, ἔχω, πόνος, χὥτʼ, ἥβη, ἐμός, εὐσθεν...","[Βρόμιος, ἔχω, πόνος, χὥτʼ, ἥβη, ἐμός, εὐσθενέ...",2535,None,"[Βρόμιος, ἔχω, πόνο*, χὥτʼ, ἥβη, ἐμός, εὐσθενέ...","[[Βρόμιος, ἔχω, πόνο*, χὥτʼ, ἥβη, ἐμός, εὐσθεν...",7,0,0,1
7,tlg0006.tlg004.perseus-grc2.xml,Euripides,Ἡρακλεῖδαι,6272,tlg0006,tlg0006.tlg004,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[[ποτός, εἰμί, οὗτος, δεδογμένον], [δίκαιος, φ...","[ποτός, εἰμί, οὗτος, δεδογμένον, δίκαιος, φύω,...",3545,None,"[ποτός, εἰμί, οὗτος, δεδογμένον, δίκαιος, φύω,...","[[ποτός, εἰμί, οὗτος, δεδογμένον], [δίκαιος, φ...",11,0,1,2
8,tlg0006.tlg005.perseus-grc2.xml,Euripides,Ἱππόλυτος,8257,tlg0006,tlg0006.tlg005,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[[πολύς, βροτός, ἀνώνυμος, θεά, καλέω, Κύπρις,...","[πολύς, βροτός, ἀνώνυμος, θεά, καλέω, Κύπρις, ...",4898,None,"[πολύς, βροτός, ἀνώνυμος, θεά, καλέω, Κύπρις, ...","[[πολύς, βροτός, ἀνώνυμος, θεά, καλέω, Κύπρις,...",8,3,8,8
9,tlg0006.tlg006.perseus-grc2.xml,Euripides,Ἀνδρομάχη,7397,tlg0006,tlg0006.tlg006,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[[Ἀσιανός, γῆ, σχῆμα, θηβαία, πόλις, ἑδνόω, πο...","[Ἀσιανός, γῆ, σχῆμα, θηβαία, πόλις, ἑδνόω, πολ...",4420,None,"[Ἀσιανός, γῆ, σχῆμα, θηβαία, πόλις, ἑδνόω, πολ...","[[Ἀσιανός, γῆ, σχῆμα, θηβαία, πόλις, ἑδνόω, πο...",5,0,7,4
10,tlg0006.tlg007.perseus-grc2.xml,Euripides,Ἑκάβη,7340,tlg0006,tlg0006.tlg007,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[[ἥκω, νεκρός, κευθμών, σκότος, πύλη, λιπάω, ᾍ...","[ἥκω, νεκρός, κευθμών, σκότος, πύλη, λιπάω, ᾍδ...",4422,None,"[ἥκω, νεκρός, κευθμών, σκότος, πύλη, λιπάω, ᾍδ...","[[ἥκω, νεκρός, κευθμών, σκότος, πύλη, λιπάω, ᾍ...",9,0,7,6
11,tlg0006.tlg008.perseus-grc2.xml,Euripides,Ἱκέτιδες,7166,tlg0006,tlg0006.tlg008,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[[Δήμητηρ, ἑστιοῦχος, Ἐλευσίς, χθών, ναός, ἔχω...","[Δήμητηρ, ἑστιοῦχος, Ἐλευσίς, χθών, ναός, ἔχω,...",4402,None,"[Δήμητηρ, ἑστιοῦχος, Ἐλευσίς, χθών, ναός, ἔχω,...","[[Δήμητηρ, ἑστιοῦχος, Ἐλευσίς, χθών, ναός, ἔχω...",18,0,8,3
12,tlg0006.tlg009.perseus-grc2.xml,Euripides,Ἡρακλῆς,7977,tlg0006,tlg0006.tlg009,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[[Ζεύς, σύλλεκτρος, οἶδα, βροτός, Ἀργεῖος, ἀμφ...","[Ζεύς, σύλλεκτρος, οἶδα, βροτός, Ἀργεῖος, ἀμφι...",4936,None,"[Ζεύς, σύλλεκτρος, οἶδα, βροτός, Ἀργεῖος, ἀμφι...","[[Ζεύς, σύλλεκτρος, οἶδα, βροτός, Ἀργεῖος, ἀμφ...",16,0,1,6
13,tlg0006.tlg010.perseus-grc2.xml,Euripides,Ἴων,9339,tlg0006,tlg0006.tlg010,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[[Ἄτλας, χαλκοῦς, νῶτον, οὐρανός, θεός, παλαιό...","[Ἄτλας, χαλκοῦς, νῶτον, οὐρανός, θεός, παλαιός...",5643,None,"[Ἄτλας, χαλκοῦς, νῶτον, οὐρανός, θεός, παλαιός...","[[Ἄτλας, χαλκοῦς, νῶτον, οὐρανός, θεός, παλαιό...",12,1,3,6
14,tlg0006.tlg011.perseus-grc2.xml,Euripides,Τρῳάδες,7230,tlg0006,tlg0006.tlg011,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[[ἥκω, λείπω, Αἰγαῖος, ἁλμυρός, βάθος, Πόντος,...","[ἥκω, λείπω, Αἰγαῖος, ἁλμυρός, βάθος, Πόντος, ...",4536,None,"[ἥκω, λείπω, Αἰγαῖος, ἁλμυρός, βάθος, Πόντος, ...","[[ἥκω, λείπω, Αἰγαῖος, ἁλμυρός, βάθος, Πόντος,...",9,0,11,5


In [15]:
authors_overview = cgl.groupby("author").sum().drop(["date_avr", "date_manual"], axis=1)
authors_overview

,wordcount,n_sentences,lemmata_wordcount,count_πόνο*,count_ὀδύν*,count_ἄλγ*,count_λύπ*
author,,,,,,,
Aeneas Tacticus,13993,745,6902,6,0,0,0
Aeschines,47136,3281,23031,2,0,1,5
Aeschines [Sp.],4273,188,2055,0,0,2,0
Alcidamas,5983,263,3016,5,0,0,1
Andocides,18987,1958,8902,1,0,0,2
Antiphon,22136,2405,9936,1,0,0,1
Antisthenes,924,61,417,0,0,0,0
Aristophanes,94897,11585,50255,21,4,13,18
Aristoteles,12571,792,5196,2,0,0,3


In [17]:
set_with_dataframe(PIPA_data.add_worksheet("authors_overview",1,1), authors_overview, include_index=True)

In [8]:
subcorpora_overview_simple = cgl.groupby("subcorpus").sum().drop(["date_avr", "date_manual"], axis=1)
subcorpora_overview_simple

,wordcount,n_sentences,lemmata_wordcount,count_πόνο*,count_ὀδύν*,count_ἄλγ*,count_λύπ*
subcorpus,,,,,,,
c_aristotelicum,785703,56737,343165,103,3,34,406
c_hippocraticum,333443,24456,171332,657,645,315,58
c_platonicum,574294,52384,272931,121,14,67,316


In [9]:
subcorpora_overview_simple.columns

Index(['wordcount', 'n_sentences', 'lemmata_wordcount', 'count_πόνο*',
       'count_ὀδύν*', 'count_ἄλγ*', 'count_λύπ*'],
      dtype='object')

In [10]:
set_with_dataframe(PIPA_data.add_worksheet("subcorpora_overview_simple", 1,1), subcorpora_overview_simple, include_index=True)

In [12]:
subcorpora_overview = cgl[cgl["subcorpus"].notnull()][['subcorpus', 'title', 'wordcount', 'n_sentences', 'lemmata_wordcount', 'count_πόνο*','count_ὀδύν*', 'count_ἄλγ*', 'count_λύπ*']]
subcorpora_overview

,subcorpus,title,wordcount,n_sentences,lemmata_wordcount,count_πόνο*,count_ὀδύν*,count_ἄλγ*,count_λύπ*
535,c_platonicum,Euthyphro,5393,659,2420,0,0,0,0
536,c_platonicum,Apology,8659,483,3990,2,0,0,3
537,c_platonicum,Crito,4268,376,1887,0,0,0,3
538,c_platonicum,Phaedo,22519,1441,9900,2,0,4,14
539,c_platonicum,Cratylus,18985,2260,8950,0,1,2,4
...,...,...,...,...,...,...,...,...,...
1086,c_hippocraticum,De habitu decenti,1518,124,713,0,0,0,0
1087,c_hippocraticum,Precepts,1351,105,733,2,0,2,2
1088,c_hippocraticum,De crisibus,2035,200,1097,5,16,3,0
1089,c_hippocraticum,De diebus criticis,1295,112,612,1,5,4,0


In [25]:
set_with_dataframe(PIPA_data.add_worksheet("subcorpora_overview", 1,1), subcorpora_overview)

In [19]:
def sentences_with_term(sentences, term, flatlist=False):
    sentences_with_term = [sen for sen in sentences if term in sen]
    if flatlist == True:
        flatlist = [word for sen in sentences_with_term for word in sen]
        return flatlist
    else: 
        return sentences_with_term

In [20]:
for word in key_words:
    cgl[word + "_sents"] = cgl["lemmatized_sentences_repl"].apply(lambda x: sentences_with_term(x, word, True))

In [21]:
cgl

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,...,lemmata_repl,lemmatized_sentences_repl,count_πόνο*,count_ὀδύν*,count_ἄλγ*,count_λύπ*,πόνο*_sents,ὀδύν*_sents,ἄλγ*_sents,λύπ*_sents
1,tlg0003.tlg001.perseus-grc2.xml,Thucydides,The Peloponnesian War,150118,tlg0003,tlg0003.tlg001,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελο...","[[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελ...",31,0,6,25,"[χρόνος, πολύς, ναυμαχία, ἐπικρατέω, θάλασσα, ...",[],"[ῥᾳθυμία, πόνο*, μελέτη, νόμος, πολύς, τρόπος,...","[εὐπραξία, σπάνιος, τὶς, πολέμιος, λύπ*, πολύς..."
6,tlg0006.tlg001.perseus-grc2.xml,Euripides,Cyclops,4141,tlg0006,tlg0006.tlg001,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[Βρόμιος, ἔχω, πόνο*, χὥτʼ, ἥβη, ἐμός, εὐσθενέ...","[[Βρόμιος, ἔχω, πόνο*, χὥτʼ, ἥβη, ἐμός, εὐσθεν...",7,0,0,1,"[Βρόμιος, ἔχω, πόνο*, χὥτʼ, ἥβη, ἐμός, εὐσθενέ...",[],[],"[ἐμπίνω, κὰμφαγεῖν, ὀπάζω, ἡμέρα, Ζεύς, οὗτος,..."
7,tlg0006.tlg004.perseus-grc2.xml,Euripides,Ἡρακλεῖδαι,6272,tlg0006,tlg0006.tlg004,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[ποτός, εἰμί, οὗτος, δεδογμένον, δίκαιος, φύω,...","[[ποτός, εἰμί, οὗτος, δεδογμένον], [δίκαιος, φ...",11,0,1,2,"[αἰδώς, συγγενής, σέβω, ἔξεστι, Ἄργος, ναίω, π...",[],"[αἰσχύνω, σός, λόγος, τύχη, ἄλγ*]","[λύω, μέλος, λύπ*, προπίτνων, θεός, φέρω, ὑπερ..."
8,tlg0006.tlg005.perseus-grc2.xml,Euripides,Ἱππόλυτος,8257,tlg0006,tlg0006.tlg005,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[πολύς, βροτός, ἀνώνυμος, θεά, καλέω, Κύπρις, ...","[[πολύς, βροτός, ἀνώνυμος, θεά, καλέω, Κύπρις,...",8,3,8,8,"[πολύς, προκόψασʼ, πόνο*, πολύς, δεῖ, εἰμί, ἁπ...","[κεφαλή, ᾄσσουσʼ, ὀδύν*, ἐγκέφαλος, πηδάω, σφά...","[ἡδύς, παῖς, ταὐτός, ἄλγ*, τάλας, ὅδε, ἄλγ*, α...","[ναυβάτης, τὶς, πλέω, κρή, ἔξορμος, ἀνήρ, λιμή..."
9,tlg0006.tlg006.perseus-grc2.xml,Euripides,Ἀνδρομάχη,7397,tlg0006,tlg0006.tlg006,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[Ἀσιανός, γῆ, σχῆμα, θηβαία, πόλις, ἑδνόω, πολ...","[[Ἀσιανός, γῆ, σχῆμα, θηβαία, πόλις, ἑδνόω, πο...",5,0,7,4,"[γυνή, Θέτις, δάπεδον, ἄναξ, θάσσω, δηρός, λεί...",[],"[σιωπή, Κύπρις, ἄλγ*, χρώς, δεινός, τραῦμα, ἄλ...","[δίδυμος, λέκτρον, ἐπαινέω, βροτός, ἀμφιμήτορε..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150,tlg1210.tlg001.1st1K-grc1.xml,Autolycus,De Sphaera Quae Movetur,4831,tlg1210,tlg1210.tlg001,4 B.C.,-3.5,{'-3.5': 1},NaN,...,"[ὁμαλός, λέγω, φέρω, σημεῖον, ὅσος, ἴσος, χρόν...","[[ὁμαλός, λέγω, φέρω, σημεῖον, ὅσος, ἴσος, χρό...",0,0,0,0,[],[],[],[]
1151,tlg1210.tlg002.1st1K-grc1.xml,Autolycus,De Ortibus et Occasibus,13388,tlg1210,tlg1210.tlg002,4 B.C.,-3.5,{'-3.5': 1},NaN,...,"[ἀπλανής, ἄστρον, ἐπιτολή, δύσις, λέγω, ἀληθιν...","[[], [ἀπλανής, ἄστρον, ἐπιτολή, δύσις, λέγω, ἀ...",0,0,0,0,[],[],[],[]
1166,tlg1409.opp003.1st1K-grc1.xml,Heraclides Ponticus,Res Publica Atheniensium,378,tlg1409,tlg1409.opp003,4 B.C.,-3.5,{'-3.5': 1},-3.5,...,"[Ἡρακλέης, πολιτεία, Ἀθήναιος, Ἀθηναῖος, εἶμι,...","[[], [], [], [], [], [Ἡρακλέης, πολιτεία, Ἀθήν...",0,0,0,0,[],[],[],[]
1179,tlg1486.tlg001.1st1K-grc1.xml,Matron of Pitana,Δει̑πνον Ἀττικόν,3632,tlg1486,tlg1486.tlg001,4 B.C.,-3.5,{'-3.5': 1},NaN,...,"[δεῖπνον, ἐνέπω, Μοῦσα, πολύτροφος, πολύς, ξεν...","[[δεῖπνον, ἐνέπω, Μοῦσα, πολύτροφος, πολύς, ξε...",0,0,0,0,[],[],[],[]


In [22]:
cgl.to_json("../data/large_data/cgl_sents.json")